In [1]:
import ipywidgets
from IPython import display
import os
import re
import glob
from tqdm import tqdm
import codecs
from chardet import detect
import pprint
from tqdm import tqdm_notebook
import random

!apt install -qq enchant
!pip install pyenchant
from enchant.checker import SpellChecker
from enchant.tokenize import EmailFilter, URLFilter

import torch

!pip install transformers
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import AutoModelForSequenceClassification
from transformers import pipeline

display.clear_output()
from google.colab import drive
drive.mount('/content/drive')
display.clear_output()
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("\nRunning on device: ", str(device).upper())

if gpu_info.find('failed') >= 0 and ram_gb < 30:
  print('\nSelect the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
  print('\nTo enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  try:
      if gpu_info.find('failed') < 0:
         print(gpu_info)
  except:
    display.clear_output()
  finally:
    print('\nYour runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
    print('You are using a high-RAM runtime!')


Running on device:  CUDA
Mon Nov 30 23:58:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    26W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                              

In [3]:
!pwd

/content


In [2]:
tokenizer = AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")
model = AutoModelForSequenceClassification.from_pretrained("joeddav/xlm-roberta-large-xnli")
generator = pipeline("zero-shot-classification", device=0, model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!cd

In [ ]:
literary_tokens = ["characterization", "character", "setting", 
                   "exposition", "climax", "resolution", "plot", 
                   "context", "action", "weapon", 
                   "danger", "death", "suspense", 
                   "emotion", "surprise", "problem", "conflict", 
                   "perspective", "transition", "relief", "metaphor", 
                   "flashback"]

subgenre_tokens = ['Vampire', 'Ghost', 'Horror', 'Comedic Horror', 'Murder', 
                   'Werewolf', 'Apocalypse','Haunted House', 'Witch', 'Hell', 
                   'Alien', 'Gore', 'Monster']

subgenre_token = {'Vampire': '[VAMPIRE]', 'Ghost' : '[GHOST]', 'Horror' : '[HORROR]', 'Comedic Horror' : '[COMEDIC HORROR]', 'Murder' : '[MURDER]', 'Werewolf' : '[WEREWOLF]', 'Apocalypse' : '[APOCALYPSE]',
              'Haunted House' : '[HAUNTED HOUSE]', 'Witch' : '[WITCH]', 'Hell' : '[HELL]', 'Alien' : '[ALIEN]', 'Gore' : '[GORE]', 'Monster' : '[MONSTER]'}

author_list = ['Brian Evenson', 'M R James', 'Ambrose Bierce', 'Elliott O Donnell', 'Joseph Sheridan Le Fanu',
               'Edgar Allan Poe', 'Bram Stoker', 'Algernon Blackwood', 
               'Madeline Gobbo', 'Nnedi Okorafor', 'Sofia Samatar', 'Franz Kafka', 
               'Laird Barron', 'Nathan Ballingrud', 'Nellie Bly', 'William Hope Hodsgon',
               'Kelly Link', 'Arthur Machen', 'George Sylvester Viereck', 'Robert Chambers', 
               'John Meade Falkner', 'Ann Radcliffe', 'Howard Lovecraft', 'Robert Louis Stevenson',
               'Edith Birkhead', 'Jeff Vandermeer', 'Henry James', 'John William Polidori', 'W Bob Holland',
               'Prest and Rymer', 'Oliver Onions', 'Stephen King', 'Clive Barker']

literary_threshold = 0.95
subgenre_threshold = 0.25

# get file encoding type
def get_encoding_type(file):
    with open(file, 'rb') as f:
        rawdata = f.read()
    return detect(rawdata)['encoding']


def correctTxtEncoding(filename, encoding_to='UTF-8'):
    from_codec = get_encoding_type(filename)
    temp_filename = filename[:-4]+"temp.txt"
    try:
        with open(filename, 'r', encoding=from_codec) as fr:
            with open(temp_filename, 'w', encoding=encoding_to) as fw:
                for line in fr:
                    fw.write(line[:-1]+'\r\n')
        os.remove(filename) # remove old encoding file
        os.rename(temp_filename, filename) # rename new encoding
    except UnicodeDecodeError:
        print('Decode Error')
    except UnicodeEncodeError:
        print('Encode Error')

def get_all_txt_files():
    path = "/content/drive/My Drive/CSCI 470 Project/Epic/Data/" +'\**\*.txt';
    files = glob.glob(path, recursive=True)
    return files

def get_files_in_data_folder(folder):
    path = "/content/drive/My Drive/CSCI 470 Project/Epic/Data/" + folder + '/*.txt';
    files = glob.glob(path, recursive=True)
    return files

def get_files_by_author(author):
    path = "/content/drive/My Drive/CSCI 470 Project/Epic/Data/" + '\**\*_' + author + '.txt';
    files = glob.glob(path, recursive=True)
    return files

def merge_texts(texts):
    merged_text = ''
    random.shuffle(texts)
    for text in tqdm(texts):
        correctTxtEncoding(text)
        with open(text,'r',encoding="utf8") as f:
            for line in f:
                merged_text += line[:-1]+'\n'
    merged_text_cleaned = clean_text(merged_text)
    return merged_text_cleaned

def get_train_test_validation(txt_file, train=0.70, test=0.20, val=0.10):
    train_doc =[]
    test_doc = []
    val_doc =[]
    with open(txt_file,'r', encoding='UTF-8') as f:
        file_input=f.readlines()

    count = 0
    for cnt, line in enumerate(file_input):
            if cnt <= len(file_input)*train:
                train_doc.append(line)
            elif (cnt > len(file_input)*train and cnt < len(file_input)*(train+test)):
                test_doc.append(line)
            else:
                val_doc.append(line)

    ## Write to file
    f = open(txt_file[:-4]+'_train.txt', "w+", encoding='UTF-8')
    count = 0
    for line in train_doc:
        count=count+1
        f.write(str(line))
        f.write("\n")  
    f.close()
    print("Training lines:\t",count)
    
    ## Write to file
    f = open(txt_file[:-4]+'_test.txt', "w+", encoding='UTF-8')
    count = 0
    for line in train_doc:
        count=count+1
        f.write(str(line))
        f.write("\n")  
    f.close()
    print("Testing lines:\t",count)

    ## Write to file
    f = open(txt_file[:-4]+'_val.txt',"w+", encoding='UTF-8')
    count = 0
    for line in val_doc:
        count=count+1
        f.write(str(line))
        f.write("\n")

    f.close()
    print("Validation lines:\t",count)

def create_train_test(txt_file, train=0.80, test=0.20):
    train_doc =[]
    test_doc = []
    val_doc =[]
    with open(txt_file,'r', encoding='UTF-8') as f:
        file_input=f.readlines()

    count = 0
    for cnt, line in enumerate(file_input):
            if cnt <= len(file_input)*train:
                train_doc.append(line)
            elif (cnt > len(file_input)*train and cnt < len(file_input)*(train+test)):
                test_doc.append(line)
            else:
                val_doc.append(line)

    ## Write to file
    f = open(txt_file[:-4]+'_train.txt', "w+", encoding='UTF-8')
    count = 0
    for line in train_doc:
        count=count+1
        f.write(str(line))
        f.write("\n")  
    f.close()
    print("Training lines:\t",count)
    
    ## Write to file
    f = open(txt_file[:-4]+'_test.txt', "w+", encoding='UTF-8')
    count = 0
    for line in train_doc:
        count=count+1
        f.write(str(line))
        f.write("\n")  
    f.close()
    print("Testing lines:\t",count)

def save_output_file(path_from_output, file_name, data):
    path = "/content/drive/My Drive/CSCI 470 Project/Epic/Data/" + path_from_output + file_name;
    with open(path, "w", encoding='UTF-8') as file:
        file.write(data)
    print("File saved at:\t", path)
    return path

def checkSpelling(text):
    chkr = SpellChecker("en_US",filters=[EmailFilter,URLFilter])
    chkr.set_text(text)
    """
    for err in chkr:
        print("ERROR:", err.word)
    """

def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

def insertTokenRegex(text, regex, token, 
                     beginning_of_match=False, 
                     beginning_of_line=False,
                     end_of_match=False, 
                     end_of_line=False):
    matches=re.findall(regex,text)
    for match in matches:
        index = text.find(match)
        if beginning_of_match:
            text = text[:index] + token + text[index:]
        if (end_of_match):
            text = text[:index+1] + match + token + text[len(match)+index+1:]
    if (len(matches) != 0):
        if (end_of_line):
            text = text + token
        if (beginning_of_line):
            text = token + text
    return text

def insertAuthorToken(text, token):
    return token + text


def clean_text(string):
    pattern = '(page|PAGE|Page)(\s+\|\s+)([0-9]+)(.*)$'
    output_cleaned = re.sub('\s$', '', string, flags=re.MULTILINE)
    p=re.compile(pattern,re.MULTILINE)
    output_cleaned = p.sub(" ",output_cleaned)
    output_cleaned = output_cleaned.replace('“','"').replace('”','"')
    return output_cleaned 

def find_num_sentences(text):
    num_sentences = 0

    #find and store locations of quotations within text 
    dialogue_locations = []
    dialogue_pattern = r'"(?:(?:(?!(?<!\\)").)*)[.?!,]"'
    for match in re.finditer(dialogue_pattern, text):
        s = match.start()
        e = match.end()
        dialogue_locations.append((s, e))
        

    for match in re.finditer("[!.?]", text):
        skip_punc = False
        punc_i = match.end() #get index of next punctuation

        #check if punctuation found was used in an honorific
        pat_obj = re.compile('(Mr)|(Mrs)|(Dr)|(Ms)|(Sr)|(Jr)|(Mt)', re.IGNORECASE)
        if pat_obj.search(text[punc_i-4: punc_i]):
            continue
        
        #if punctuation found is in between a set of quote, skip iteration       
        for s, e in dialogue_locations:
            if (s <= punc_i) and (punc_i <= e):
                skip_punc = True
                break
        if skip_punc == True:
            skip_punc = False
            continue 
        num_sentences += 1

    return num_sentences

def add_tokens_to_text(line, author_token, generator, movie=False):
    punc_pattern = '[.!?]'
    tokens = ''

    #classify the lines according to literary tokens
    literary_generator = generator(line, literary_tokens, multi_class=True)
    for i, score in enumerate(literary_generator['scores']):
        if score > literary_threshold:
            tokens += "["+ literary_generator['labels'][i].upper() + "]"
        else: break

    
    #classify the lines according to subgenre tokens
    subgenre_generator = generator(line, subgenre_tokens, multi_class=False)
    if subgenre_generator['scores'][0] > subgenre_threshold:
        tokens += subgenre_token[subgenre_generator['labels'][0]]
    

    if movie:
        dialogue_pattern = r'[A-Z]+:'
        for match in re.finditer(dialogue_pattern, line):
            s = match.start()
            line = line[:s] + '[DIALOGUE]' + line[s:]
    else: 
        line = insertTokenRegex(line, r'"(?:(?:(?!(?<!\\)").)*)"', "[DIALOGUE]", beginning_of_match=True)

    line = tokens + line
    line = insertTokenRegex(line, r'((?i)Chapter\s+[0-9MDCLXVI]*.*)', "[CHAPTER]", beginning_of_match=True)
    if author_token != None: line = insertAuthorToken(line, author_token)

    return line + "\n"

def clean_files_sequential(file_paths):
    print("Loading Zero Shot Tokenization Model Pipeline\n"+"="*80)
    
    tokenizer = AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")
    model = AutoModelForSequenceClassification.from_pretrained("joeddav/xlm-roberta-large-xnli")
    generator = pipeline("zero-shot-classification", device=0, model=model, tokenizer=tokenizer)
    display.clear_output()
    
    output_path = "/content/drive/My Drive/CSCI 470 Project/Epic/Data/Cleaned_Text/"
    movie_input_path = "/content/drive/My Drive/CSCI 470 Project/Epic/Data/Horror_Movie_Transcripts"
    print("Input files: ")
    for txt in file_paths:
        print(txt)
    print("Output files: ")
    for txt in file_paths:
        print(output_path+txt.split("/")[-1])

    for i, txt_file in enumerate(file_paths):
        display.clear_output(wait=True)
        output_path_name = txt_file.split("/")[-1]
        print("Working on Cleaning "+ output_path_name)
        print(str(i)+"/"+str(len(file_paths)) + "\tFiles Completed")
        """
        Check to see if file already exists to continue cleaning for previous spot.
        """
        start_line = 0
        author_token = None
        files = glob.glob(output_path+"*"+txt_file.split("/")[-1])
        if len(files) > 0: 
            write_mode = "a"
            with open(files[0],'r',encoding="utf8") as clean_reader:
                start_line = len(clean_reader.readlines())
        else:
            """
            File doesn't already exist so setup the cleaning
            """
            write_mode = "w" # Overwrite current file
            correctTxtEncoding(txt_file)
            files = glob.glob(movie_input_path+txt_file.split("/")[-1])
            if len(files) > 0: author_token = '[MOVIE]'
            else:
                for author in author_list:
                    if txt_file.replace("_", " ").find(author) != -1:
                        author_token = "["+ author.upper() + "]"
                        break    
        
        """
        Start cleaning of THIS txt file
        """
        with open(txt_file,'r', encoding="utf8") as original_reader:
            with open(output_path+output_path_name, write_mode, encoding="utf8") as clean_writer:
                full_txt = original_reader.readlines()
                full_length = len(full_txt)
                with tqdm(total=full_length, position=start_line, leave=True) as pbar:
                    if author_token == '[MOVIE]':
                        for line in tqdm(range(start_line, full_length), position=start_line, leave=True):
                            pbar.update()
                            new_text_cleaned = clean_text(full_txt[line][:-1])+'\n'
                            tokenized_txt = add_tokens_to_text(new_text_cleaned, author_token, generator, movie=True)
                            clean_writer.write(tokenized_txt)
                    else:
                        sequence = ""
                        for line in tqdm(range(start_line, full_length), position=start_line, leave=True):
                            pbar.update()
                            new_text_cleaned = clean_text(full_txt[line][:-1]+" ")
                            sequence += new_text_cleaned
                            if find_num_sentences(sequence) > 3:
                                tokenized_txt = add_tokens_to_text(sequence, author_token, generator)
                                clean_writer.write(tokenized_txt)
                                sequence = ""


    display.clear_output()
    print("FINISHED CLEANING\n"+"="*80)
    print("Output files: ")
    for txt in file_paths:
        print("\t"+output_path+txt.split("/")[-1])

# all_text_files = get_all_txt_files()
# harry_potter_texts = merge_texts(get_files_in_data_folder("Harry_Potter"))
# print("Cleaned Harry Potter Text:\n\n\n" + harry_potter_texts[:2000]+"....\n\n")
# stephen_king_texts = merge_texts(get_files_by_author("Stephen_King"))
# print("Cleaned Stephen Text:\n\n\n" + stephen_king_texts[:2000]+"....\n\n")
# horror_movie_transcripts = merge_texts(get_files_in_data_folder("Horror_Movie_Transcripts"))
# print("Cleaned Horror Movie Transcripts:\n\n\n" + horror_movie_transcripts[:2000]+"....\n\n")
# public_domain_texts = merge_texts(get_files_in_data_folder("Public_Domain_Horror_Novels"))
# print("Cleaned Public Domain Horror Novels:\n\n\n" + public_domain_texts[:2000]+"....\n\n")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/CSCI 470 Project/Epic/Data/Black_List.txt"
newpath = "/content/drive/My Drive/CSCI 470 Project/Epic/Data/Cleaned_Black_List.txt"
black_list_words = set()
with open(path, "r") as reader:
    line = reader.readline()
    words = line.split(",")
    for word in words:
        black_list_words.add(word)
with open(newpath, "w") as writer:
    for word in words:
        writer.write(word+"\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
clean_files_sequential(get_files_in_data_folder("Stephen_King"))
clean_files_sequential(get_files_in_data_folder("Horror_Movie_Transcripts"))
clean_files_sequential(get_files_in_data_folder("Public_Domain_Horror_Novels"))
clean_files_sequential(get_files_in_data_folder("Harry_Potter"))
clean_files_sequential(get_files_in_data_folder("Clive_Barker"))
clean_files_sequential(get_files_in_data_folder("Mary_Shelly"))

FINISHED CLEANING
Output files: 
	/content/drive/My Drive/CSCI 470 Project/Epic/Data/Cleaned_Text/Carrie_Stephen_King.txt
	/content/drive/My Drive/CSCI 470 Project/Epic/Data/Cleaned_Text/The_Dead_Zone_Stephen_King.txt
	/content/drive/My Drive/CSCI 470 Project/Epic/Data/Cleaned_Text/Misery_Stephen_King.txt
	/content/drive/My Drive/CSCI 470 Project/Epic/Data/Cleaned_Text/The_Shining_Stephen_King.txt
	/content/drive/My Drive/CSCI 470 Project/Epic/Data/Cleaned_Text/Black_House_Stephen_King.txt
	/content/drive/My Drive/CSCI 470 Project/Epic/Data/Cleaned_Text/It_Stephen_King.txt
	/content/drive/My Drive/CSCI 470 Project/Epic/Data/Cleaned_Text/Salems_Lot_Stephen_King.txt
	/content/drive/My Drive/CSCI 470 Project/Epic/Data/Cleaned_Text/Bag_of_Bones_Stephen_King.txt


In [ ]:
create_train_test(save_output_file('Tokenized/',
                                    'merged_tokenized.txt',
                                    merge_texts(get_files_in_data_folder("Cleaned_Text"))))


100%|██████████| 95/95 [03:59<00:00,  2.53s/it]


File saved at:	 /content/drive/My Drive/CSCI 470 Project/Epic/Data/Tokenized/merged_tokenized.txt
Training lines:	 81135
Testing lines:	 81135


In [ ]:
# from transformers import TextDataset,DataCollatorForLanguageModeling

# def load_dataset(train_path,test_path,tokenizer):
#     train_dataset = TextDataset(
#           tokenizer=tokenizer,
#           file_path=train_path,
#           block_size=128)

#     test_dataset = TextDataset(
#           tokenizer=tokenizer,
#           file_path=test_path,
#           block_size=128)

#     data_collator = DataCollatorForLanguageModeling(
#         tokenizer=tokenizer, mlm=False,
#     )
#     return train_dataset,test_dataset,data_collator

# train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)